In [ ]:
from time import time
import numpy as np
import numexpr as ne
import iarray as ia
from numba import jit

In [ ]:
NSLICES = 50
SLICE_THICKNESS = 10
NTHREADS = 4
CLEVEL = 5
CLIB = ia.LZ4
BLOCKSIZE = 0
# CLEVEL = 1
# CLIB = ia.ZSTD
cparams = dict(clevel=CLEVEL, clib=CLIB, nthreads=NTHREADS, blocksize=BLOCKSIZE)

In [ ]:
# The expressions to compute
sexpr = "(sin(x) - 3.2) * (cos(x) + 1.2)"
sexpr2 = "(x - y) * (z - 3.) * (y - x - 2)"

In [ ]:
def open_datafile(filename):
    dataset = ia.from_file(filename, load_in_mem=False)
    return dataset
t0 = time()
precipitation = open_datafile("ia-data/rea6/tot_prec/2018.iarray")
t1 = time()
print("Time to open file: %.3f" % (t1 - t0))
#print("dataset:", precipitation)

In [ ]:
# Get a random number of slices
nt, nx, ny = precipitation.shape
shape = (NSLICES * SLICE_THICKNESS, nx, ny)
pshape = (1, nx, ny)
# tslices = np.random.choice(nt - SLICE_THICKNESS, NSLICES)

In [ ]:
def concatenate_slices(dataset, tslice):
    dtshape = ia.dtshape(shape=shape, pshape=pshape, dtype=np.float32)
    iarr = ia.empty(dtshape, **cparams)
    for i, (_, precip_block) in enumerate(iarr.iter_write_block()):
        slice_ia = dataset[slice(tslice + i, tslice + i + 1), :, :]
        slice_np = ia.iarray2numpy(slice_ia)
        precip_block[:, :, :] = slice_np
    return iarr

In [ ]:
np.random.seed(1)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec1 = concatenate_slices(precipitation, tslice)
t1 = time()
print("Time for concatenating %d slices into an ia container (1): %.3f" % (NSLICES * SLICE_THICKNESS, (t1 - t0)))
print("cratio", prec1.cratio)

In [ ]:
np.random.seed(2)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec2 = concatenate_slices(precipitation, tslice)
t1 = time()
print("Time for concatenating %d slices into an ia container (2): %.3f" % (NSLICES * SLICE_THICKNESS, (t1 - t0)))
print("cratio", prec2.cratio)

In [ ]:
np.random.seed(3)
tslice = np.random.choice(nt - NSLICES * SLICE_THICKNESS)
t0 = time()
prec3 = concatenate_slices(precipitation, tslice)
t1 = time()
print("Time for concatenating %d slices into an ia container (3): %.3f" % (NSLICES * SLICE_THICKNESS, (t1 - t0)))
print("cratio", prec3.cratio)

In [ ]:
def compute_slices(x):
    expr = ia.Expr(eval_flags="iterblock", **cparams)
    expr.bind("x", x)
    expr.compile(sexpr)
    out = expr.eval(shape, pshape, x.dtype)
    return out
t0 = time()
slices_computed = compute_slices(prec1)
t1 = time()
print("Time for computing '%s' expression with 1 operand (iarray): %.3f" % (sexpr, (t1 - t0)))

In [ ]:
np_prec1 = ia.iarray2numpy(prec1)
def compute_numpy(x):
    out = (np.sin(x) - 3.2) * (np.cos(x) + 1.2)
    return out
t0 = time()
npslices_computed = compute_numpy(np_prec1)
t1 = time()
print("Time for computing '%s' expression with 1 operand (via numpy): %.3f" % (sexpr, (t1 - t0)))

In [ ]:
def compute_slices2(dset1, dset2, dset3):
    expr = ia.Expr(eval_flags="iterblock", **cparams)
    expr.bind("x", dset1)
    expr.bind("y", dset2)
    expr.bind("z", dset3)
    expr.compile(sexpr2)
    out = expr.eval(shape, pshape, dset1.dtype)
    return out
t0 = time()
slices_computed2 = compute_slices2(prec1, prec2, prec3)
t1 = time()
print("Time for computing '%s' expression with 3 operands (iarray): %.3f" % (sexpr2, (t1 - t0)))

In [ ]:
np_prec1 = ia.iarray2numpy(prec1)
np_prec2 = ia.iarray2numpy(prec2)
np_prec3 = ia.iarray2numpy(prec3)
def compute_numpy2(x, y, z):
    out = (x - y) * (z - 3.) * (y - x - 2)
    return out
t0 = time()
npslices_computed = compute_numpy2(np_prec1, np_prec2, np_prec3)
t1 = time()
print("Time for computing '%s' expression with 3 operands (via numpy): %.3f" % (sexpr2, (t1 - t0)))